<font size=20>Annotated by Size Final Version</font>

In [52]:
import sys
print(sys.version)
import pandas as pd
import nltk
%matplotlib inline
import math
import json
from ast import literal_eval
import numpy as np
import re
from matplotlib import pyplot as plt
from colour import Color
from IPython.core.display import HTML
from matplotlib import cm
from matplotlib.colors import rgb2hex
plt.rcParams["figure.figsize"] = [16, 6]

3.8.8 (default, Apr 13 2021, 12:59:45) 
[Clang 10.0.0 ]


In [53]:
# Set up all your file paths here!
matches = '/Users/annie/GIT/quotation-detection/text_matcher/fanondata.jsonl'
originalText = '/Users/annie/Downloads/frantz-fanon-richard-philcox-jean-paul-sartre-homi-k.-bhabha-the-wretched-of-the-earth-grove-press-2011.txt'

# Replace with the format: AuthorLastName_TitleOfText (ex: Butler_GenderTrouble)
textname = 'Fanon_WretchedOfTheEarth'

In [54]:
with open(matches) as f: 
    rawMatches = f.readlines()

data = [json.loads(line) for line in rawMatches]
df = pd.DataFrame(data)

In [55]:
# Adapted from text-matcher
class Text: 
    def __init__(self, filename): 
        self.filename = filename
        tokenizer = nltk.RegexpTokenizer('[a-zA-Z]\w+\'?\w*') # A custom regex tokenizer. 
        spans = list(tokenizer.span_tokenize(self.text))
        # Take note of how many spans there are in the text
        self.length = spans[-1][-1] 
        
    @property
    def text(self):
        """ Reads the file in memory. """
        f = open(self.filename, encoding='utf-8', errors='ignore')
        return f.read() 

    @property
    def tokens(self, removeStopwords=True): 
        """ Tokenizes the text, breaking it up into words, removing punctuation. """
        tokenizer = nltk.RegexpTokenizer('[a-zA-Z]\w+\'?\w*') # A custom regex tokenizer. 
        spans = list(tokenizer.span_tokenize(self.text))
        # Take note of how many spans there are in the text
        self.length = spans[-1][-1] 
        tokens = tokenizer.tokenize(self.text)
        tokens = [ token.lower() for token in tokens ] # make them lowercase
        if not removeStopwords: 
            self.spans = spans
            return tokens
        tokenSpans = list(zip(tokens, spans)) # zip it up
        stopwords = nltk.corpus.stopwords.words('english') # get stopwords
        tokenSpans = [ token for token in tokenSpans if token[0] not in stopwords ] # remove stopwords from zip
        self.spans = [ x[1] for x in tokenSpans ] # unzip; get spans
        return [ x[0] for x in tokenSpans ] # unzip; get tokens

In [56]:
text = Text(originalText)

In [57]:
# Get the size of the text. 
textALength = text.length

# Make an empty array the size of the text. 
tally = np.zeros(textALength, dtype=np.int)

<ipython-input-57-672b2f69b7ad>:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  tally = np.zeros(textALength, dtype=np.int)


In [58]:
# Read the locations from the CSV file, and literally evaluate them into lists. 
locations = df['Locations in A']

In [59]:
# Tally up every time a letter in the novel is quoted. 
for article in locations: 
    for locRange in article: 
         for i in range(locRange[0], min(locRange[1]+1, len(tally))):
                tally[i] += 1

In [60]:
# Make a color list in hex for all the values in the tally. 
# Let's hope there aren't too many. 
colors = list(np.arange(5,75,70/(tally.max()+1)))
colorList = colors

In [61]:
# Create a CSS Stylesheet for each color value in the map. 
colorCSS = ""
for i, color in zip(range(0, tally.max()+1), colorList): 
    colorCSS += ".c-%s { font-size: %spx; }\n" % (i, color)

In [62]:
n = 50

checkpoints = np.linspace(0, textALength, n).round()
checkpoints = [int(point) for point in checkpoints]

In [63]:
def span(val): 
    return '<span class="c-%s">' % val

previousVal = None
for i, valChar in enumerate(zip(tally, text.text)):
    val, char = valChar[0], valChar[1]
    if previousVal == None: 
        # First character. 
        out = '<span class="c-%s">' % val
    elif val != previousVal: 
        out += '</span><span class="c-%s">' % val
    if i in checkpoints: 
        out += '<a name="b-%s"></a>' % checkpoints.index(i)
    out += char
    previousVal = val

In [64]:
html = """<!DOCTYPE html>
<html>
<head>
  <link href="https://fonts.googleapis.com/css?family=Raleway" rel="stylesheet"> 
  <style>
  pre { 
      font-family: Raleway, serif; 
    }
  main { 
      width: 40em; 
      margin: 2em auto; 
  }
  %s
  </style>
  """ % (colorCSS)
html += """
  </head>
  <body><main><pre>%s</pre></main></body></html>
  """ % (out)

In [65]:
with open(f'{textname}_Annotated_by_Size.html', 'w') as f: 
    f.write(html)
    f.close()